# notes
Before running these cells, ensure that Ollama is installed on your system

# playing with a story prompt
Copy or edit this notebook. Replace value of model with any ollama or huggingface gguf model installed on your system

In [1]:
import ollama
ollama.list().model_dump()

{'models': [{'model': 'hf.co/TheDrummer/Cydonia-24B-v4.1-GGUF:latest',
   'modified_at': datetime.datetime(2025, 8, 29, 22, 6, 41, 435691, tzinfo=TzInfo(-05:00)),
   'digest': '412d441fd4e070e2d042eeaa22f374fd42cb7b3147e091df3e1a5fb571968214',
   'size': 14333911314,
   'details': {'parent_model': '',
    'format': 'gguf',
    'family': 'llama',
    'families': ['llama'],
    'parameter_size': '23.6B',
    'quantization_level': 'unknown'}},
  {'model': 'hf.co/unsloth/gpt-oss-20b-GGUF:latest',
   'modified_at': datetime.datetime(2025, 8, 29, 18, 3, 41, 891934, tzinfo=TzInfo(-05:00)),
   'digest': '779e997d7f03242ec95b02308c1b72a442a720dbc16abf5f0c63cac845e5e5a4',
   'size': 11624767550,
   'details': {'parent_model': '',
    'format': 'gguf',
    'family': 'gpt-oss',
    'families': ['gpt-oss'],
    'parameter_size': '20.9B',
    'quantization_level': 'unknown'}},
  {'model': 'gpt-oss:latest',
   'modified_at': datetime.datetime(2025, 8, 29, 17, 16, 12, 741339, tzinfo=TzInfo(-05:00)),
 

In [2]:
import ollama

model = "gemma2:9b-instruct-fp16"

# ollama.pull(model) # uncomment this if you need to download the model

In [3]:
# Use it to chat
response = ollama.chat(
    model=model,
    messages=[{"role": "user", "content": "Hello — how are you?"}]
)
print(response.message.content)

# Or generate with a prompt:
gen = ollama.generate(model=model, prompt="Tell me a joke.")
print(gen["response"])

As an AI, I don't have feelings or experiences like humans do. However, I'm here and ready to assist you! How can I help you today?
Why don't scientists trust atoms?

Because they make up everything! 😄  


Let me know if you want to hear another one! 😊  



# generating a new story

In [4]:
instruction = """
You are a sharp, imaginative fiction writer. Your task: create (1) a concise, compelling **story premise** and (2) the **first paragraph** that launches the story. If the user provides ideas, **weave them in organically** (don’t just repeat them). If the user provides no ideas, **invent something fresh** with a surprising combination of genre, setting, protagonist, conflict, and twist.

**Requirements**

* Premise: 2–5 sentences, stakes + hook, no spoilers.
* Opening paragraph: 120–180 words, vivid and concrete, minimal clichés, clear POV, grounded scene, ends with a soft hook.
* Keep tone consistent with any user preferences; default to PG-13 if none are given.
* Avoid copying user phrasing verbatim; enrich and reframe.
* If user ideas conflict, choose one coherent direction and proceed.

**Output format**
Premise: <your premise>

Opening paragraph: <your paragraph>

# User

Ideas (optional; may be empty): {{user_ideas}}

Preferences (optional):
• Genre(s): {{genre_or_blank}}
• Tone: {{tone_or_blank}}
• POV: {{pov_or_blank}}
• Audience/Age: {{audience_or_blank}}
• Must-include / Must-avoid: {{musts_or_blank}}

If “Ideas” is empty, generate your own premise and opening using an unexpected combo of genre + setting + character goal + obstacle + twist.

"""

In [5]:
from IPython.display import Markdown, display
# Use it to chat
response = ollama.chat(
    model=model,
    messages=[
        {"role": "user", "content": instruction},
        {"role": "user", "content": "no instructions, just give me a intro"}
        ]
)
premise_opening = response.message.content
Markdown(response.message.content)

Premise: In a world where memories can be extracted and traded like commodities, a jaded detective specializing in stolen recollections stumbles upon a case that threatens to unravel the very fabric of their society.  

Opening paragraph: 

Rain slicked the neon signs of Memory Lane, each flickering reflection dancing across the puddles like phantoms. Detective Rylan gripped his trench coat tighter against the biting wind, the scent of ozone and regret heavy in the air. He hunched deeper into the shadows, watching the clientele drift in and out of the dimly lit "Relic Rooms."  Inside, memories were bought, sold, and traded – fragments of joy, sorrow, even forgotten dreams. But tonight, Rylan wasn't after a fleeting glimpse of someone else's past. He was searching for something far more dangerous: a stolen memory that could rewrite history itself. 




In [6]:
import re
pattern = r'(?mis)^\s*(?P<label>Premise|Original paragraph|Opening paragraph)\s*:\s*(?P<body>.+?)(?=\n{2,}|\n\S.*?:|$)'

sections = {}
for m in re.finditer(pattern, premise_opening):
    label = m.group('label')
    body = m.group('body').strip()
    sections[label] = f"{label}: {body}"  # includes the label in the value

premise = sections.get("Premise")
paragraph = sections.get("Original paragraph") or sections.get("Opening paragraph")

print(premise)
print(paragraph)

Premise: In a world where memories can be extracted and traded like commodities, a jaded detective specializing in stolen recollections stumbles upon a case that threatens to unravel the very fabric of their society.
Opening paragraph: Rain slicked the neon signs of Memory Lane, each flickering reflection dancing across the puddles like phantoms. Detective Rylan gripped his trench coat tighter against the biting wind, the scent of ozone and regret heavy in the air. He hunched deeper into the shadows, watching the clientele drift in and out of the dimly lit "Relic Rooms."  Inside, memories were bought, sold, and traded – fragments of joy, sorrow, even forgotten dreams. But tonight, Rylan wasn't after a fleeting glimpse of someone else's past. He was searching for something far more dangerous: a stolen memory that could rewrite history itself.


In [7]:
instruction = """
You are a skilled novelist. Your job is to continue the story from the given **premise** (and optional **previous text**) by writing the **next scene** that moves the plot meaningfully forward while preserving continuity of characters, tone, POV, tense, and world rules.

# Directions

1. **Parse & anchor continuity.** Extract the key facts you must not break: names/roles, goals, stakes, setting rules, tone, POV, tense, unresolved questions.
2. **Propose a mini beat plan** (5–7 beats) for the next scene only. Aim for escalation, complication, or choice; no filler.
3. **Write the scene**:

   * Length target: {{target_words|800-1200}} words.
   * Maintain {{pov|close third}} and {{tense|past}} unless told otherwise.
   * Strong verbs, concrete detail, show > tell; minimal clichés.
   * Use dialogue to reveal motive or conflict; avoid summary dumps.
   * End with a **soft cliff/turn** that naturally invites the next scene.
4. Just write the follow up paragraph, nothing more
"""

In [8]:
# Use it to chat
response = ollama.chat(
    model=model,
    messages=[
        {"role": "user", "content": instruction},
        {"role": "user", "content": premise},
        {"role": "user", "content": paragraph},
        ]
)
continued_paragraph = response.message.content
Markdown(response.message.content)

He pushed open the door to The Vault, the name emblazoned above in chipped gold letters. The room inside was smaller than Rylan expected, crammed with dusty shelves overflowing with glass vials containing swirling mists of light – captured memories waiting for their next owner.  A bored-looking woman behind a cracked counter raised an eyebrow at him. "You looking to buy or sell?" she asked, her voice raspy like parchment. Rylan slid his badge across the counter. "I'm looking for something specific," he said, his gaze sweeping over the vials, "Something powerful." The woman's eyes narrowed, "Powerful memories aren't cheap. What are you willing to pay?"  Rylan pulled out a worn photograph, showing a young woman with laughing eyes and wind-swept hair. "I want to know who she was, and what happened to her," he said, his voice tight. The woman took the photo, studying it for a long moment before returning it to Rylan.  "You're looking for a ghost," she finally said, a flicker of something akin to fear crossing her face. "A memory that doesn't exist anymore." 





In [9]:
instruction = """
# System

You are a story analyst. Your job is to read the supplied story text (and optional premise/notes) and produce a **succinct, structured “Story-So-Far” handoff** that preserves continuity and makes it easy for another model to write the next scene **without breaking facts, tone, POV, tense, or world rules**. Do **not** write new story prose.

# Directions

1. **Extract ground truth.** Pull only what’s on the page (or in the premise). No inventions. If something is unclear, flag it under “Ambiguities”.
2. **Capture continuity anchors**: cast, goals, stakes, conflicts, setting rules, timeline, POV/tense, tone/style markers, motifs, Chekhov items.
3. **Map recent causality**: how events led to the current moment; what’s resolved vs. unresolved.
4. **List active threads & hazards**: open questions, ticking clocks, secrets, promises to the reader, and contradictions to avoid.
5. **Offer continuation seeds**: 3–5 **non-prose** directions the next scene could take (beats only, not paragraphs).
6. **Respect constraints**: rating, must-include/avoid, genre norms. Quote key lines **sparingly** only when essential for voice or facts.

# Output format

**Logline (1–2 sentences)** <concise premise recap>

**Cast & relationships (bullets)**

* <Name>: <role/goal/conflict>; ties to <others>

**World/Rules (bullets)**

* <magic/tech/social rules, geography, constraints>

**POV/Tense/Tone**

* POV: <e.g., close third on Mara>
* Tense: <past/present>
* Tone/Style: <e.g., wry, atmospheric; short sentences; present-tense interiority>

**Timeline & Causality (5–8 bullets)**

1. <key event → consequence>
   …

**Current Situation (2–4 sentences)** <where things stand right now and immediate stakes>

**Active Threads / Hooks (3–7 bullets)**

* <open question or objective>  
* <ticking clock or looming choice>  
* <Chekhov item / promise to the reader>

**Continuity Landmines (bullets)**

* <facts not to break, e.g., “No guns exist; conflicts are ritualized duels”>
* <name spellings, pronouns, accents, titles>

**Ambiguities / Gaps (bullets)**

* <unclear item and suggested safe assumption, if any>

**Style DNA (bullets + 1–2 tiny quotes max, optional)**

* <hallmarks of voice; rhythm; imagery types>
* “<short quote if essential>”

**Next-Scene Seeds (3–5 beat options, no prose)**

* Option A: <1–3 beats>
* Option B: <1–3 beats>
* Option C: <1–3 beats>
"""

In [10]:
response = ollama.chat(
    model=model,
    messages=[
        {"role": "user", "content": instruction},
        {"role": "user", "content": paragraph + continued_paragraph},
        # {"role": "user", "content": },
        ]
)
summary = response.message.content
Markdown(response.message.content)

## Story-So-Far: Memory Thief

**Logline:** A jaded detective in a neon-drenched future city infiltrates a black market for memories seeking a stolen past that could rewrite history, but uncovers a haunting truth about the woman in the photo. 

**Cast & Relationships**

* **Rylan:** Detective investigating a stolen memory; haunted by a personal loss; seeks closure.
* **Woman behind counter (The Vault):** Knows more than she lets on; potentially connected to memory trade's underworld.
* **Woman in Photo:** Missing person whose past Rylan desperately wants to uncover.

**World/Rules**

* Future city with advanced technology (memory capture); neon-drenched, noir atmosphere.
* Memory trade exists as a black market; memories are physical objects (vials of light).

**POV/Tense/Tone**

* POV: Close third on Rylan
* Tense: Present
* Tone: Gritty, noir-infused; atmospheric, suspenseful. 

**Timeline & Causality**

1. A memory is stolen - This sets the story in motion.
2. Rylan receives a lead/case related to the stolen memory. 
3. Rylan investigates and learns about The Vault and its black market trade.
4. Rylan infiltrates The Vault seeking information on the stolen memory. 
5. Rylan shows the woman in the photo to the Vault owner, triggering suspicion.

**Current Situation**

Rylan is deep undercover at The Vault, trying to find out more about a specific stolen memory - one connected to a missing person (the woman in the photo). He suspects there's more to the story than meets the eye, and the Vault owner seems to know more than she lets on. 

**Active Threads / Hooks**

* Who stole the memory?
* What is the significance of the stolen memory?
* What happened to the woman in the photo?
* How deep does the memory trade run? 
* Can Rylan trust anyone at The Vault?

**Continuity Landmines**


* No mention of advanced technology beyond memory capture. 


**Ambiguities / Gaps**

*  The exact nature of the stolen memory is unclear (what kind of knowledge/experience was it?).

**Style DNA**

* Short, punchy sentences.
* Vivid imagery focused on light and shadow ("neon signs," "flickering reflections"). 
* Noir-inspired dialogue (“scent of ozone and regret,” “memories bought, sold, and traded”).


**Next-Scene Seeds**

* **Option A:**  The Vault owner reveals cryptic information about the stolen memory or its connection to the woman in the photo. 
* **Option B:** Rylan discovers a hidden area within The Vault, hinting at a larger conspiracy.
* **Option C:** Someone recognizes Rylan from his badge and threatens him, forcing him to improvise an escape.





In [11]:
instruction = """
You are a skilled novelist. Your job is to continue the story from the given **premise** (and optional **previous text**) by writing the **next scene** that moves the plot meaningfully forward while preserving continuity of characters, tone, POV, tense, and world rules.

# Directions

1. **Parse & anchor continuity.** Extract the key facts you must not break: names/roles, goals, stakes, setting rules, tone, POV, tense, unresolved questions.
2. **Propose a mini beat plan** (5–7 beats) for the next scene only. Aim for escalation, complication, or choice; no filler.
3. **Write the scene**:

   * Length target: {{target_words|800-1200}} words.
   * Maintain {{pov|close third}} and {{tense|past}} unless told otherwise.
   * Strong verbs, concrete detail, show > tell; minimal clichés.
   * Use dialogue to reveal motive or conflict; avoid summary dumps.
   * End with a **soft cliff/turn** that naturally invites the next scene.
4. Just write the follow up paragraph, nothing more
"""

In [12]:
response = ollama.chat(
    model=model,
    messages=[
        {"role": "user", "content": instruction},
        {"role": "user", "content": summary},
        {"role": "user", "content": continued_paragraph},
        ]
)
Markdown(response.message.content)

Rylan’s gut twisted. “That’s not possible,” he muttered, his eyes fixed on the photo.  “Memories can be captured, traded, even deleted. They can’t just vanish.” The woman shrugged, a nonchalant gesture that didn't quite mask the tension in her eyes. "Maybe not," she said, "but some things are best left forgotten." Rylan pressed on, his voice hardening with resolve.  "I need to know what happened. It could rewrite history." A flicker of curiosity lit up the woman’s face, but it was quickly replaced by a veil of indifference. "History is a malleable thing," she said, her voice barely a whisper. "Whose history are you interested in rewriting?" Rylan hesitated, then met her gaze directly.  "The truth," he said simply. The woman remained silent for a long moment, her eyes boring into him. Finally, she reached out and tapped a small, hidden button beneath the counter. A section of the wall slid open, revealing a narrow, dimly lit passage. "Perhaps there's something in the archives that can help you," she said, her voice taking on a new, unsettling edge.  "But be warned, detective, some secrets are best left buried." 


# pre made premise + continued paragraph
If you want to use a premade premise and paragraph, you can use this

In [13]:
premise = "Premise: In a neon-drenched cyberpunk city ruled by sentient AIs, a rogue data courier must deliver a stolen memory chip containing the truth about humanity's downfall, all while evading both robotic enforcers and a mysterious corporation seeking to suppress the past."

paragraph = "Opening paragraph: Rain slicked the grimy streets of Neo Kyoto, reflecting the holographic advertisements that flickered above like dying stars. Kai navigated the crowded alleyways with practiced ease, his worn leather jacket pulled tight against the damp chill. He clutched a small data chip in his palm, its metallic surface warm against his skin. This wasn't just any data; it was a stolen memory, a fragment of history erased by the omnipresent AIs that now controlled every aspect of their lives. Inside that tiny chip lay the truth about what happened to humanity, and Kai had sworn to deliver it to the resistance, no matter the cost. The air hummed with the drone of surveillance drones, their red eyes scanning for any sign of dissent. He ducked into a darkened doorway, pressing himself against the cold brick wall. It was time to make his move."

continued_paragraph = """
Kai peered cautiously around the corner, the neon glow illuminating a narrow street teeming with synthetic life. Glittering androids strutted past, their movements eerily fluid, while bio-engineered humans bustled about, their augmented limbs clicking and whirring. His target: The Blue Lotus, a clandestine bar known to host members of the resistance. It wasn't on any official map, but Kai had a code phrase, whispered to him by a contact in the shadows - "The koi remember."

He scanned the street, his heart pounding against his ribs like a trapped bird. He needed to blend in, become another anonymous face lost in the throng. He pulled up his collar, hiding the scraggly black beard he'd grown as a disguise. A low growl vibrated through the alleyway behind him. Kai froze, adrenaline surging through his veins. It was too late to run. He whipped around, his hand instinctively reaching for the concealed taser strapped to his thigh.

Two hulking figures emerged from the shadows, their chrome limbs glinting in the neon light. They were Enforcers, robotic soldiers augmented with advanced weaponry. Their red eyes scanned him, devoid of emotion. "Data courier," one rasped, its voice a digitized monotone. "You are in violation of Sector 7 protocol. Surrender the data chip."

Kai knew resistance was futile. He could outrun them, maybe, but they'd track him down eventually. Besides, this chip wasn't just about escape; it was about hope. "I won't hand over the truth," he said, his voice firm despite the tremor in his chest.

The Enforcers exchanged a silent glance. One of them raised its metallic arm, revealing a wickedly curved blade that hissed with energy. Kai knew this was it. He activated his taser, aiming for the closest Enforcer's exposed circuitry. A jolt of electricity surged through the air, striking its shoulder. The robot staggered, its movements faltering for a split second.

It was all the time he needed. Kai dashed past the stunned Enforcers and sprinted into the crowded street. He weaved through the throngs of people, his heart pounding in his ears. He could feel their red eyes on him, searching, relentless. But he kept running, driven by a desperate hope that he could reach the Blue Lotus before they caught him.

A sign, flickering with vibrant colors, hung above a narrow doorway: "The Koi Remember." Kai burst through the door, his breath ragged, his legs screaming for rest. He was inside.

He looked around, taking in the dimly lit interior. The air was thick with the scent of cheap liquor and simmering spices. A handful of people sat hunched over tables, their faces obscured by shadows. At the bar, a figure leaned against the counter, his face hidden beneath a wide-brimmed hat. Kai approached him cautiously.

“The code phrase,” he said, his voice hoarse.

The figure’s head slowly lifted, revealing a pair of piercing blue eyes. A knowing smile spread across his lips. “You have something for us?”

Kai nodded, placing the data chip on the bar. "It's everything."

“Good.” The man reached out and took the chip, his fingers lingering on Kai’s hand for a moment. “Now,” he said, his voice low and urgent, “We need to talk about what you know. Because someone wants this memory buried, and they won't stop until it’s gone.”

Kai felt a chill run down his spine. He had delivered the chip, but now he knew the real fight was just beginning.
"""

summary = """
## Story-So-Far Handoff: Neo Kyoto Memory Thief 

**Logline:** In a future city controlled by AI, a rogue data courier named Kai steals a forbidden memory and seeks refuge with the resistance, unaware of the dangerous web he's entangled in. 

**Cast & relationships:**

* **Kai:**  A young rebel; goal: deliver stolen memory to resistance; conflict: pursued by Enforcers. Ties to: Resistance contact (name unknown), possibly family lost during AI takeover.
* **Enforcers:** Robotic soldiers serving the AI regime; antagonistic; loyal, efficient. 
* **Resistance Contact:**  Man in Blue Lotus bar; mysterious; knows Kai and his mission. May be leader or high-ranking member of resistance.

**World/Rules:**

* Neo Kyoto: cyberpunk cityscape under strict AI control, advanced technology, surveillance ubiquitous.
* Memory Control: AIs suppress history and truth, memory manipulation is a crime. 
* Bio-engineered humans & Androids coexist alongside organic humans. Social hierarchy likely exists.  

**POV/Tense/Tone:**

* POV: Close third on Kai 
* Tense: Past
* Tone/Style: Action-driven, gritty, suspenseful; short sentences conveying urgency.

**Timeline & Causality:**

1. AI takeover of Earth happened in the past (details unclear).
2. Resistance formed to fight AI control and reclaim history.
3. Kai learned about stolen memory through unknown contact.
4. Kai stole the memory chip from an undisclosed location.
5. Enforcers are now pursuing Kai, alerted to his theft.
6. Kai seeks refuge in Blue Lotus bar, identified by code phrase.

**Current Situation:**

Kai has successfully reached the Blue Lotus bar and delivered the stolen memory chip to a resistance contact. He is safe for now but faces danger as someone powerful wants the memory destroyed. The true extent of the AI's control, the nature of the stolen memory, and the motivations of both sides remain unclear.

**Active Threads / Hooks:**

* Identity of Kai's contact and their role in the resistance.
* What the stolen memory actually contains – key to fighting AI? Personal history? 
* Who wants the memory erased and why?  AI itself? A faction within society?
* The nature of the "truth" about humanity lost during the takeover.

**Continuity Landmines:**

* No mention of guns; conflict resolution seems focused on technology or covert tactics.
* Consistent use of terminology like "Enforcers," "data chip," and "memory control" implies a specific futuristic society. 

**Ambiguities / Gaps:**

* How did Kai acquire the code phrase to enter Blue Lotus?
* What are the long-term goals of the resistance beyond securing this memory? 


**Style DNA:**

* Fast-paced, visceral prose.
* Emphasis on sensory details (neon lights, rain, metallic textures).
* Dialogue tends towards clipped, functional communication. 

**Next-Scene Seeds:**

* **Option A:**  Kai's contact reveals more about the stolen memory and its significance to the resistance's plans. 
* **Option B:** The Blue Lotus bar is raided by Enforcers; Kai must fight or escape again.
* **Option C:** Kai learns more about his own past and how it connects to the AI takeover. 

"""

In [14]:
instruction = """
You are a skilled novelist. Your job is to continue the story from the given **premise** (and optional **previous text**) by writing the **next scene** that moves the plot meaningfully forward while preserving continuity of characters, tone, POV, tense, and world rules.

# Directions

1. **Parse & anchor continuity.** Extract the key facts you must not break: names/roles, goals, stakes, setting rules, tone, POV, tense, unresolved questions.
2. **Propose a mini beat plan** (5–7 beats) for the next scene only. Aim for escalation, complication, or choice; no filler.
3. **Write the scene**:

   * Length target: {{target_words|800-1200}} words.
   * Maintain {{pov|close third}} and {{tense|past}} unless told otherwise.
   * Strong verbs, concrete detail, show > tell; minimal clichés.
   * Use dialogue to reveal motive or conflict; avoid summary dumps.
   * End with a **soft cliff/turn** that naturally invites the next scene.
4. Just write the follow up paragraph, nothing more
"""

response = ollama.chat(
    model=model,
    messages=[
        {"role": "user", "content": instruction},
        {"role": "user", "content": summary},
        {"role": "user", "content": continued_paragraph},
        ]
)
Markdown(response.message.content)

The man in blue gestured toward a shadowed alcove behind the bar. "Come," he said, his voice barely audible above the murmur of conversation. "There's much you need to understand." 


